In [13]:
import pandas as pd
import os
from os.path import join
from sklearn.model_selection import train_test_split
import torch
import numpy as np

BASE = '/data/ephemeral/home/data'
TABLES = join(BASE, 'tables_V2.0')
FEATURES = '/data/ephemeral/home/data/base_features'

front_view_only = False

# 데이터 병합

### 데이터 불러오기

In [2]:
ad_extra = pd.read_csv(join(TABLES, 'Ad_table (extra).csv'))
ad_extra.rename(columns={' Genmodel_ID':'Genmodel_ID', ' Genmodel':'Genmodel'}, inplace=True)
ad_extra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268255 entries, 0 to 268254
Data columns (total 24 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Maker         268255 non-null  object 
 1   Genmodel      268255 non-null  object 
 2   Genmodel_ID   268255 non-null  object 
 3   Adv_ID        268255 non-null  object 
 4   Adv_year      268255 non-null  int64  
 5   Adv_month     268255 non-null  int64  
 6   Color         246380 non-null  object 
 7   Reg_year      268248 non-null  float64
 8   Bodytype      267301 non-null  object 
 9   Runned_Miles  267200 non-null  object 
 10  Engin_size    266191 non-null  object 
 11  Gearbox       268088 non-null  object 
 12  Fuel_type     267846 non-null  object 
 13  Price         267110 non-null  float64
 14  Engine_power  236444 non-null  float64
 15  Annual_Tax    221580 non-null  object 
 16  Wheelbase     240257 non-null  float64
 17  Height        240454 non-null  float64
 18  Widt

In [3]:
ad_extra.columns

Index(['Maker', 'Genmodel', 'Genmodel_ID', 'Adv_ID', 'Adv_year', 'Adv_month',
       'Color', 'Reg_year', 'Bodytype', 'Runned_Miles', 'Engin_size',
       'Gearbox', 'Fuel_type', 'Price', 'Engine_power', 'Annual_Tax',
       'Wheelbase', 'Height', 'Width', 'Length', 'Average_mpg', 'Top_speed',
       'Seat_num', 'Door_num'],
      dtype='object')

In [4]:
image_data = pd.read_csv(join(TABLES, 'Image_table.csv'))
image_data.rename(columns={' Image_ID': 'Image_ID', ' Image_name': 'Image_name', ' Predicted_viewpoint':'Predicted_viewpoint', ' Quality_check':'Quality_check'}, inplace=True)
image_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1451784 entries, 0 to 1451783
Data columns (total 5 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   Genmodel_ID          1451784 non-null  object
 1   Image_ID             1451784 non-null  object
 2   Image_name           1451784 non-null  object
 3   Predicted_viewpoint  1451784 non-null  int64 
 4   Quality_check        86993 non-null    object
dtypes: int64(1), object(4)
memory usage: 55.4+ MB


In [5]:
price_data = pd.read_csv(join(TABLES, 'Price_table.csv'))
price_data.rename(columns={' Genmodel': 'Genmodel', ' Genmodel_ID': 'Genmodel_ID', ' Year': 'Year', ' Entry_price': 'Entry_price'}, inplace=True)
price_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6333 entries, 0 to 6332
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Maker        6333 non-null   object
 1   Genmodel     6333 non-null   object
 2   Genmodel_ID  6333 non-null   object
 3   Year         6333 non-null   int64 
 4   Entry_price  6333 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 247.5+ KB


In [6]:
sales_data = pd.read_csv(join(TABLES, 'Sales_table.csv'))
sales_data.rename(columns={'Genmodel ': 'Genmodel', 'Genmodel ID  ': 'Genmodel_ID'}, inplace=True)
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 773 entries, 0 to 772
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Maker        773 non-null    object
 1   Genmodel     773 non-null    object
 2   Genmodel_ID  773 non-null    object
 3   2020         773 non-null    int64 
 4   2019         773 non-null    int64 
 5   2018         773 non-null    int64 
 6   2017         773 non-null    int64 
 7   2016         773 non-null    int64 
 8   2015         773 non-null    int64 
 9   2014         773 non-null    int64 
 10  2013         773 non-null    int64 
 11  2012         773 non-null    int64 
 12  2011         773 non-null    int64 
 13  2010         773 non-null    int64 
 14  2009         773 non-null    int64 
 15  2008         773 non-null    int64 
 16  2007         773 non-null    int64 
 17  2006         773 non-null    int64 
 18  2005         773 non-null    int64 
 19  2004         773 non-null    

### Ad_extra: Engine_size 변환

In [7]:
def extract_engine_size(x):
    # 값이 문자열인지 확인
    if isinstance(x['Engin_size'], str):
        return float(x['Engin_size'][:-1])  # 마지막 문자 제거 후 float 변환
    else:
        return x['Engin_size']

ad_extra['Engine_size'] = ad_extra.apply(lambda x: extract_engine_size(x), axis=1)
ad_extra.drop(columns=['Engin_size'], inplace=True)
ad_extra

,Maker,Genmodel,Genmodel_ID,Adv_ID,Adv_year,Adv_month,Color,Reg_year,Bodytype,Runned_Miles,...,Annual_Tax,Wheelbase,Height,Width,Length,Average_mpg,Top_speed,Seat_num,Door_num,Engine_size
0,Bentley,Arnage,10_1,10_1$$1,2018,4,Silver,2000.0,Saloon,60000,...,NaN,3116.0,1515.0,2125.0,5390.0,NaN,NaN,5.0,4.0,6.8
1,Bentley,Arnage,10_1,10_1$$2,2018,6,Grey,2002.0,Saloon,44000,...,315,3116.0,1515.0,2125.0,5390.0,13.7 mpg,179 mph,5.0,4.0,6.8
2,Bentley,Arnage,10_1,10_1$$3,2017,11,Blue,2002.0,Saloon,55000,...,315,3116.0,1515.0,2125.0,5390.0,14.7 mpg,155 mph,5.0,4.0,6.8
3,Bentley,Arnage,10_1,10_1$$4,2018,4,Green,2003.0,Saloon,14000,...,NaN,3116.0,1515.0,2125.0,5390.0,NaN,NaN,5.0,4.0,6.8
4,Bentley,Arnage,10_1,10_1$$5,2017,11,Grey,2003.0,Saloon,61652,...,NaN,3116.0,1515.0,2125.0,5390.0,NaN,NaN,5.0,4.0,6.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268250,Westfield,Sport,97_1,97_1$$1,2018,5,Yellow,2006.0,Convertible,1800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.2
268251,Westfield,Sport,97_1,97_1$$2,2018,5,Yellow,2006.0,Convertible,2009,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
268252,Zenos,E10,99_1,99_1$$1,2018,3,Red,2016.0,Convertible,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.0
268253,Zenos,E10,99_1,99_1$$2,2018,3,Green,2016.0,Convertible,1538,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.0


### Adding missing values to physical table ad_extra table (추가)

Ad_table (extra).csv 파일에서 결측치(NaN)나 0 값을 채우기 위해, 동일한 Genmodel_ID를 가진 다른 행의 값을 활용하여 데이터를 보완

In [8]:
ad_extra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268255 entries, 0 to 268254
Data columns (total 24 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Maker         268255 non-null  object 
 1   Genmodel      268255 non-null  object 
 2   Genmodel_ID   268255 non-null  object 
 3   Adv_ID        268255 non-null  object 
 4   Adv_year      268255 non-null  int64  
 5   Adv_month     268255 non-null  int64  
 6   Color         246380 non-null  object 
 7   Reg_year      268248 non-null  float64
 8   Bodytype      267301 non-null  object 
 9   Runned_Miles  267200 non-null  object 
 10  Gearbox       268088 non-null  object 
 11  Fuel_type     267846 non-null  object 
 12  Price         267110 non-null  float64
 13  Engine_power  236444 non-null  float64
 14  Annual_Tax    221580 non-null  object 
 15  Wheelbase     240257 non-null  float64
 16  Height        240454 non-null  float64
 17  Width         240175 non-null  float64
 18  Leng

In [9]:
# Manual touches

# Peugeot RCZ
ad_extra.loc[ad_extra['Genmodel_ID'] == '69_36','Wheelbase']=2612
# Ford Grand C-Max
ad_extra.loc[ad_extra['Genmodel_ID'] == '29_20','Wheelbase']=2788 

def fill_from_other_entry(row):
    for attr in ['Wheelbase', 'Length', 'Width', 'Height']:
        if pd.isna(row[attr]) or row[attr]==0:
            other_rows = ad_extra.loc[ad_extra['Genmodel_ID']==row['Genmodel_ID']]
            other_rows.dropna(subset=[attr], inplace=True)
            other_rows.drop_duplicates(subset=[attr], inplace=True)
            other_rows = other_rows[other_rows[attr]>0]
            if len(other_rows)>0:
                row[attr] = other_rows[attr].values[0]
    return row

ad_extra = ad_extra.apply(fill_from_other_entry, axis=1)

In [10]:
ad_extra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268255 entries, 0 to 268254
Data columns (total 24 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Maker         268255 non-null  object 
 1   Genmodel      268255 non-null  object 
 2   Genmodel_ID   268255 non-null  object 
 3   Adv_ID        268255 non-null  object 
 4   Adv_year      268255 non-null  int64  
 5   Adv_month     268255 non-null  int64  
 6   Color         246380 non-null  object 
 7   Reg_year      268248 non-null  float64
 8   Bodytype      267301 non-null  object 
 9   Runned_Miles  267200 non-null  object 
 10  Gearbox       268088 non-null  object 
 11  Fuel_type     267846 non-null  object 
 12  Price         267110 non-null  float64
 13  Engine_power  236444 non-null  float64
 14  Annual_Tax    221580 non-null  object 
 15  Wheelbase     263703 non-null  float64
 16  Height        267259 non-null  float64
 17  Width         267393 non-null  float64
 18  Leng

### Image_data: Adv_ID 컬럼 추가

In [11]:
'''
parser_adv_id(x)
: image_data 데이터프레임의 Image_ID 열을 '$$' 기준으로 나눔
'''
def parser_adv_id(x):
  split = x["Image_ID"].split('$$')
  return f"{split[0]}$${split[1]}"

# Image_data에 Adv_ID 컬럼 생성
image_data["Adv_ID"] = image_data.apply(lambda x: parser_adv_id(x), axis=1)
image_data.drop_duplicates(subset=['Adv_ID'], inplace=True)
image_data

,Genmodel_ID,Image_ID,Image_name,Predicted_viewpoint,Quality_check,Adv_ID
0,2_1,2_1$$1$$1,Abarth$$124 Spider$$2017$$Blue$$2_1$$1$$image_...,45,NaN,2_1$$1
1,2_1,2_1$$10$$11,Abarth$$124 Spider$$2017$$Blue$$2_1$$10$$image...,45,NaN,2_1$$10
8,2_1,2_1$$4$$0,Abarth$$124 Spider$$2017$$Blue$$2_1$$4$$image_...,0,P,2_1$$4
14,2_1,2_1$$8$$3,Abarth$$124 Spider$$2017$$Blue$$2_1$$8$$image_...,0,NaN,2_1$$8
18,2_1,2_1$$13$$8,Abarth$$124 Spider$$2017$$Grey$$2_1$$13$$image...,0,P,2_1$$13
...,...,...,...,...,...,...
1451766,96_18,96_18$$919$$3,Volvo$$XC90$$2019$$White$$96_18$$919$$image_3.jpg,225,NaN,96_18$$919
1451771,97_1,97_1$$1$$1,Westfield$$Sport$$2006$$Yellow$$97_1$$1$$image...,45,NaN,97_1$$1
1451772,99_1,99_1$$2$$14,Zenos$$E10$$2016$$Green$$99_1$$2$$image_14.jpg,180,NaN,99_1$$2
1451775,99_1,99_1$$3$$1,Zenos$$E10$$2016$$Grey$$99_1$$3$$image_1.jpg,0,P,99_1$$3


In [12]:
adv_unique = image_data['Adv_ID'].unique()
print(adv_unique)
print(len(adv_unique))

['2_1$$1' '2_1$$10' '2_1$$4' ... '99_1$$2' '99_1$$3' '99_1$$1']
247236


### Sales_data 
##### Sales_data: 첫 출시연도 컬럼 추가

In [13]:
# 첫 출시년도 찾기
def get_first_release_year(row):
    for year in range(2001, 2021, 1): 
        if row[str(year)] > 0:
            return year
    return None  # 첫 출시 년도가 없는 경우

# 'First Release Year' 컬럼 추가
sales_data['First_release_year'] = sales_data.apply(get_first_release_year, axis=1)

# NaN을 0으로 대체한 후, 정수형으로 변환
sales_data['First_release_year'] = sales_data['First_release_year'].fillna(0).astype(int)

##### Sales_data: Year 컬럼 추가
sales_data를 melt하여 연도를 열로 변환

In [14]:
# sales_data를 melt하여 연도를 열로 변환
sales_data_melted = sales_data.melt(
    id_vars=['Maker', 'Genmodel', 'Genmodel_ID', 'First_release_year'],  # 고정할 열
    var_name='Year',                              # 변환된 연도 열 이름
    value_name='Sales_amt'                        # 변환된 값 열 이름
)

# Year를 숫자형으로 변환 (병합 조건 맞추기 위해)
sales_data_melted['Year'] = sales_data_melted['Year'].astype(int)
sales_data_melted['First_release_year'] = sales_data_melted['First_release_year'].astype(int)

### 병합: Price_data + Ad_extra

In [15]:
# ad_data의 Genmodel_ID, Reg_year, price_data의 Genmodel_ID, Year 모두 동일한 경우에 데이터 병합
merged_price_ad = ad_extra.merge(price_data[['Genmodel_ID', 'Entry_price', 'Year']], left_on=['Genmodel_ID','Reg_year'], right_on=['Genmodel_ID','Year'])
print(len(merged_price_ad))
merged_price_ad

224724


,Maker,Genmodel,Genmodel_ID,Adv_ID,Adv_year,Adv_month,Color,Reg_year,Bodytype,Runned_Miles,...,Height,Width,Length,Average_mpg,Top_speed,Seat_num,Door_num,Engine_size,Entry_price,Year
0,Bentley,Arnage,10_1,10_1$$1,2018,4,Silver,2000.0,Saloon,60000,...,1515.0,2125.0,5390.0,NaN,NaN,5.0,4.0,6.8,145000,2000
1,Bentley,Arnage,10_1,10_1$$2,2018,6,Grey,2002.0,Saloon,44000,...,1515.0,2125.0,5390.0,13.7 mpg,179 mph,5.0,4.0,6.8,149000,2002
2,Bentley,Arnage,10_1,10_1$$3,2017,11,Blue,2002.0,Saloon,55000,...,1515.0,2125.0,5390.0,14.7 mpg,155 mph,5.0,4.0,6.8,149000,2002
3,Bentley,Arnage,10_1,10_1$$4,2018,4,Green,2003.0,Saloon,14000,...,1515.0,2125.0,5390.0,NaN,NaN,5.0,4.0,6.8,151500,2003
4,Bentley,Arnage,10_1,10_1$$5,2017,11,Grey,2003.0,Saloon,61652,...,1515.0,2125.0,5390.0,NaN,NaN,5.0,4.0,6.8,151500,2003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224719,Volvo,V50,96_9,96_9$$522,2018,7,Grey,2008.0,Estate,140000,...,1457.0,1770.0,4522.0,48.7 mpg,127 mph,5.0,5.0,2.0,15780,2008
224720,Volvo,V50,96_9,96_9$$523,2018,8,Blue,2007.0,Estate,158000,...,1457.0,1770.0,4522.0,40.4 mpg,137 mph,5.0,5.0,2.4,15780,2007
224721,Volvo,V50,96_9,96_9$$524,2018,5,Silver,2009.0,Estate,94000,...,1457.0,1770.0,4522.0,40.4 mpg,137 mph,5.0,5.0,2.4,15770,2009
224722,Volvo,V50,96_9,96_9$$525,2018,5,Silver,2004.0,Estate,111000,...,1452.0,1770.0,4514.0,31.0 mpg,134 mph,5.0,5.0,2.4,17165,2004


In [16]:
adv_unique = merged_price_ad['Adv_ID'].unique()
print(adv_unique)
print(len(adv_unique))

['10_1$$1' '10_1$$2' '10_1$$3' ... '96_9$$524' '96_9$$525' '96_9$$526']
224724


### 병합: Price_data + Ad_extra + Image_data

In [17]:
merged_price_ad_image = merged_price_ad.merge(image_data[['Adv_ID', 'Image_name', 'Predicted_viewpoint']], left_on=['Adv_ID'], right_on=['Adv_ID'])
merged_price_ad_image

,Maker,Genmodel,Genmodel_ID,Adv_ID,Adv_year,Adv_month,Color,Reg_year,Bodytype,Runned_Miles,...,Length,Average_mpg,Top_speed,Seat_num,Door_num,Engine_size,Entry_price,Year,Image_name,Predicted_viewpoint
0,Bentley,Arnage,10_1,10_1$$1,2018,4,Silver,2000.0,Saloon,60000,...,5390.0,NaN,NaN,5.0,4.0,6.8,145000,2000,Bentley$$Arnage$$2000$$Silver$$10_1$$1$$image_...,45
1,Bentley,Arnage,10_1,10_1$$2,2018,6,Grey,2002.0,Saloon,44000,...,5390.0,13.7 mpg,179 mph,5.0,4.0,6.8,149000,2002,Bentley$$Arnage$$2002$$Grey$$10_1$$2$$image_0.jpg,0
2,Bentley,Arnage,10_1,10_1$$3,2017,11,Blue,2002.0,Saloon,55000,...,5390.0,14.7 mpg,155 mph,5.0,4.0,6.8,149000,2002,Bentley$$Arnage$$2002$$Blue$$10_1$$3$$image_0.jpg,90
3,Bentley,Arnage,10_1,10_1$$4,2018,4,Green,2003.0,Saloon,14000,...,5390.0,NaN,NaN,5.0,4.0,6.8,151500,2003,Bentley$$Arnage$$2003$$Green$$10_1$$4$$image_0...,315
4,Bentley,Arnage,10_1,10_1$$5,2017,11,Grey,2003.0,Saloon,61652,...,5390.0,NaN,NaN,5.0,4.0,6.8,151500,2003,Bentley$$Arnage$$2003$$Grey$$10_1$$5$$image_10...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209030,Volvo,V50,96_9,96_9$$521,2018,4,Silver,2007.0,Estate,88452,...,4514.0,48.7 mpg,127 mph,5.0,5.0,2.0,15780,2007,Volvo$$V50$$2007$$Silver$$96_9$$521$$image_0.jpg,225
209031,Volvo,V50,96_9,96_9$$523,2018,8,Blue,2007.0,Estate,158000,...,4522.0,40.4 mpg,137 mph,5.0,5.0,2.4,15780,2007,Volvo$$V50$$2007$$Blue$$96_9$$523$$image_0.jpg,315
209032,Volvo,V50,96_9,96_9$$524,2018,5,Silver,2009.0,Estate,94000,...,4522.0,40.4 mpg,137 mph,5.0,5.0,2.4,15770,2009,Volvo$$V50$$2009$$Silver$$96_9$$524$$image_0.jpg,135
209033,Volvo,V50,96_9,96_9$$525,2018,5,Silver,2004.0,Estate,111000,...,4514.0,31.0 mpg,134 mph,5.0,5.0,2.4,17165,2004,Volvo$$V50$$2004$$Silver$$96_9$$525$$image_0.jpg,135


In [18]:
adv_unique = merged_price_ad_image['Adv_ID'].unique()
print(adv_unique)
print(len(adv_unique))

['10_1$$1' '10_1$$2' '10_1$$3' ... '96_9$$524' '96_9$$525' '96_9$$526']
209035


### 데이터 수정: Price_data + Ad_extra + Image_data
##### 필요없는 컬럼 삭제
Adv_year, Adv_month, Reg_year, Runned_Miles, Price, Annual_Tax, Average_mpg, Top_speed

In [19]:
columns_to_drop = ['Adv_year', 'Adv_month', 'Reg_year', 'Runned_Miles', 'Price', 'Annual_Tax', 'Average_mpg', 'Top_speed', 'Engine_power']
merged_data = merged_price_ad_image.drop(columns=columns_to_drop)
merged_data

,Maker,Genmodel,Genmodel_ID,Adv_ID,Color,Bodytype,Gearbox,Fuel_type,Wheelbase,Height,Width,Length,Seat_num,Door_num,Engine_size,Entry_price,Year,Image_name,Predicted_viewpoint
0,Bentley,Arnage,10_1,10_1$$1,Silver,Saloon,Automatic,Petrol,3116.0,1515.0,2125.0,5390.0,5.0,4.0,6.8,145000,2000,Bentley$$Arnage$$2000$$Silver$$10_1$$1$$image_...,45
1,Bentley,Arnage,10_1,10_1$$2,Grey,Saloon,Automatic,Petrol,3116.0,1515.0,2125.0,5390.0,5.0,4.0,6.8,149000,2002,Bentley$$Arnage$$2002$$Grey$$10_1$$2$$image_0.jpg,0
2,Bentley,Arnage,10_1,10_1$$3,Blue,Saloon,Automatic,Petrol,3116.0,1515.0,2125.0,5390.0,5.0,4.0,6.8,149000,2002,Bentley$$Arnage$$2002$$Blue$$10_1$$3$$image_0.jpg,90
3,Bentley,Arnage,10_1,10_1$$4,Green,Saloon,Automatic,Petrol,3116.0,1515.0,2125.0,5390.0,5.0,4.0,6.8,151500,2003,Bentley$$Arnage$$2003$$Green$$10_1$$4$$image_0...,315
4,Bentley,Arnage,10_1,10_1$$5,Grey,Saloon,Automatic,Petrol,3116.0,1515.0,2125.0,5390.0,5.0,4.0,6.8,151500,2003,Bentley$$Arnage$$2003$$Grey$$10_1$$5$$image_10...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209030,Volvo,V50,96_9,96_9$$521,Silver,Estate,Manual,Diesel,2640.0,1452.0,1770.0,4514.0,5.0,5.0,2.0,15780,2007,Volvo$$V50$$2007$$Silver$$96_9$$521$$image_0.jpg,225
209031,Volvo,V50,96_9,96_9$$523,Blue,Estate,Automatic,Diesel,2640.0,1457.0,1770.0,4522.0,5.0,5.0,2.4,15780,2007,Volvo$$V50$$2007$$Blue$$96_9$$523$$image_0.jpg,315
209032,Volvo,V50,96_9,96_9$$524,Silver,Estate,Automatic,Diesel,2640.0,1457.0,1770.0,4522.0,5.0,5.0,2.4,15770,2009,Volvo$$V50$$2009$$Silver$$96_9$$524$$image_0.jpg,135
209033,Volvo,V50,96_9,96_9$$525,Silver,Estate,Automatic,Petrol,2640.0,1452.0,1770.0,4514.0,5.0,5.0,2.4,17165,2004,Volvo$$V50$$2004$$Silver$$96_9$$525$$image_0.jpg,135


##### Adv_ID 컬럼명 수정
Adv_ID -> ID

In [20]:
merged_data.rename(columns={'Adv_ID':'ID'}, inplace=True)

##### NA 값 삭제

In [21]:
merged_data.dropna(inplace=True)
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185061 entries, 0 to 209034
Data columns (total 19 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Maker                185061 non-null  object 
 1   Genmodel             185061 non-null  object 
 2   Genmodel_ID          185061 non-null  object 
 3   ID                   185061 non-null  object 
 4   Color                185061 non-null  object 
 5   Bodytype             185061 non-null  object 
 6   Gearbox              185061 non-null  object 
 7   Fuel_type            185061 non-null  object 
 8   Wheelbase            185061 non-null  float64
 9   Height               185061 non-null  float64
 10  Width                185061 non-null  float64
 11  Length               185061 non-null  float64
 12  Seat_num             185061 non-null  float64
 13  Door_num             185061 non-null  float64
 14  Engine_size          185061 non-null  float64
 15  Entry_price          1

In [22]:
adv_unique = merged_data['ID'].unique()
print(adv_unique)
print(len(adv_unique))

['10_1$$1' '10_1$$2' '10_1$$3' ... '96_9$$524' '96_9$$525' '96_9$$526']
185061


### 병합: Price_data + Ad_extra + Image_data + Sales_data

In [23]:
# merged_df와 sales_df_melted를 병합
merged_df_sales = merged_data.merge(
    sales_data_melted,
    how="left",  # 병합 방법을 필요에 맞게 조정
    left_on=['Genmodel_ID', 'Year'],
    right_on=['Genmodel_ID', 'Year']
)

# merged_df_sales에서 ID 컬럼이 유니크하게 유지되도록 중복된 행 제거
merged_df_sales = merged_df_sales.drop_duplicates(subset=['ID'])

# 컬럼 정리
merged_df_sales.rename(columns={'Maker_x' : 'Maker', 'Genmodel_x' : 'Genmodel'}, inplace=True) 
merged_df_sales.drop(columns=['Maker_y', 'Genmodel_y' ], inplace=True)

# NA값 drop
merged_df_sales.dropna(inplace=True)

# 결과 확인
print(merged_df_sales.head())

     Maker Genmodel Genmodel_ID       ID  Color Bodytype    Gearbox Fuel_type  \
1  Bentley   Arnage        10_1  10_1$$2   Grey   Saloon  Automatic    Petrol   
2  Bentley   Arnage        10_1  10_1$$3   Blue   Saloon  Automatic    Petrol   
3  Bentley   Arnage        10_1  10_1$$4  Green   Saloon  Automatic    Petrol   
4  Bentley   Arnage        10_1  10_1$$5   Grey   Saloon  Automatic    Petrol   
5  Bentley   Arnage        10_1  10_1$$6   Blue   Saloon  Automatic    Petrol   

   Wheelbase  Height  ...  Length  Seat_num  Door_num  Engine_size  \
1     3116.0  1515.0  ...  5390.0       5.0       4.0          6.8   
2     3116.0  1515.0  ...  5390.0       5.0       4.0          6.8   
3     3116.0  1515.0  ...  5390.0       5.0       4.0          6.8   
4     3116.0  1515.0  ...  5390.0       5.0       4.0          6.8   
5     3116.0  1515.0  ...  5390.0       5.0       4.0          6.8   

   Entry_price  Year                                         Image_name  \
1       149000  2

In [24]:
merged_df_sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 176738 entries, 1 to 206316
Data columns (total 21 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Maker                176738 non-null  object 
 1   Genmodel             176738 non-null  object 
 2   Genmodel_ID          176738 non-null  object 
 3   ID                   176738 non-null  object 
 4   Color                176738 non-null  object 
 5   Bodytype             176738 non-null  object 
 6   Gearbox              176738 non-null  object 
 7   Fuel_type            176738 non-null  object 
 8   Wheelbase            176738 non-null  float64
 9   Height               176738 non-null  float64
 10  Width                176738 non-null  float64
 11  Length               176738 non-null  float64
 12  Seat_num             176738 non-null  float64
 13  Door_num             176738 non-null  float64
 14  Engine_size          176738 non-null  float64
 15  Entry_price          1

In [25]:
# sales_amt(판매량) 없는 값 삭제
merged_df_sales = merged_df_sales[merged_df_sales["Sales_amt"] != 0]

In [26]:
merged_df_sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 173471 entries, 1 to 206316
Data columns (total 21 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Maker                173471 non-null  object 
 1   Genmodel             173471 non-null  object 
 2   Genmodel_ID          173471 non-null  object 
 3   ID                   173471 non-null  object 
 4   Color                173471 non-null  object 
 5   Bodytype             173471 non-null  object 
 6   Gearbox              173471 non-null  object 
 7   Fuel_type            173471 non-null  object 
 8   Wheelbase            173471 non-null  float64
 9   Height               173471 non-null  float64
 10  Width                173471 non-null  float64
 11  Length               173471 non-null  float64
 12  Seat_num             173471 non-null  float64
 13  Door_num             173471 non-null  float64
 14  Engine_size          173471 non-null  float64
 15  Entry_price          1

In [27]:
merged_df_sales['ID'].is_unique

True

In [28]:
merged_df_sales.to_csv(join(TABLES, 'base_merged_data_2.csv'), index=False, encoding='utf-8-sig')

# 데이터 전처리

In [14]:
data_df = pd.read_csv(join(TABLES, 'base_merged_data_2.csv'))
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173471 entries, 0 to 173470
Data columns (total 21 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Maker                173471 non-null  object 
 1   Genmodel             173471 non-null  object 
 2   Genmodel_ID          173471 non-null  object 
 3   ID                   173471 non-null  object 
 4   Color                173471 non-null  object 
 5   Bodytype             173471 non-null  object 
 6   Gearbox              173471 non-null  object 
 7   Fuel_type            173471 non-null  object 
 8   Wheelbase            173471 non-null  float64
 9   Height               173471 non-null  float64
 10  Width                173471 non-null  float64
 11  Length               173471 non-null  float64
 12  Seat_num             173471 non-null  float64
 13  Door_num             173471 non-null  float64
 14  Engine_size          173471 non-null  float64
 15  Entry_price      

In [15]:
'''
conf_matrix_from_matrices(mat_gt, mat_pred)
: Confusion Matrix, 행렬 기반 데이터로부터 TP, FP, FN, TN 계산

  Input - mat_gt: Ground Truth Matrix (정답 테이블 행렬)
        - mat_pred: Predicted Matrix (모델이 예측한 결과 행렬)
'''
def conf_matrix_from_matrices(mat_gt, mat_pred):
  overlap_and = (mat_pred & mat_gt)
  tp = overlap_and.sum()
  fp = mat_pred.sum()-overlap_and.sum()
  fn = mat_gt.sum()-overlap_and.sum()
  tn = mat_gt.shape[0]**2-(tp+fp+fn)
  return tp, fp, fn, tn

In [16]:
'''
check_or_save(obj, path, index=None, header=None)
: 데이터 지정된 경로 저장 or 이미 저장된 데이터와 입력 데이터 동일한지 비교
'''
def check_or_save(obj, path, index=None, header=None):
  if isinstance(obj, pd.DataFrame):
    if index is None or header is None:
      raise ValueError('Index and header must be specified for saving a dataframe')
    if os.path.exists(path):
      if not header:
        saved_df = pd.read_csv(path,header=None)
      else:
        saved_df = pd.read_csv(path)
      naked_df = saved_df.reset_index(drop=True)
      naked_df.columns = range(naked_df.shape[1])
      naked_obj = obj.reset_index(drop=not index)
      naked_obj.columns = range(naked_obj.shape[1])
      if naked_df.round(6).equals(naked_obj.round(6)):
        return
      else:
        diff = (naked_df.round(6) == naked_obj.round(6))
        diff[naked_df.isnull()] = naked_df.isnull() & naked_obj.isnull()
        assert diff.all().all(), "Dataframe is not the same as saved dataframe"
    else:
      obj.to_csv(path, index=index, header=header)
  else:
    if os.path.exists(path):
      saved_obj = torch.load(path)
      if isinstance(obj, list):
        for i in range(len(obj)):
          check_array_equality(obj[i], saved_obj[i])
      else:
        check_array_equality(obj, saved_obj)
    else:
      print(f'Saving to {path}')
      torch.save(obj, path)

'''
check_array_equality(ob1, ob2)
: 두 배열이나 텐서가 동일한지 확인, 데이터 요소 간의 값의 일치 여부 검사
'''
def check_array_equality(ob1, ob2):
  if torch.is_tensor(ob1) or isinstance(ob1, np.ndarray):
    assert (ob2 == ob1).all()
  else:
    assert ob2 == ob1

### 연속형/범주형 데이터 전처리

In [17]:
id_df = data_df.loc[:,'ID']
image_name_df = data_df.loc[:,'Image_name']
viewpoint_df = data_df.loc[:,'Predicted_viewpoint']

# 연속형 데아터
continuous_ids = [
  'Wheelbase',
  'Height',
  'Width',
  'Length',
  'Seat_num',
  'Door_num',
  'Entry_price',
  'Year',
  'First_release_year',
  'Sales_amt',
  'Engine_size'
]

continuous_df = data_df.loc[:,continuous_ids]

# 범주형 데이터
# Genmodel_ID도 범주형 데이터로 처리함
categorical_ids = ['Maker',
  'Genmodel',
  'Genmodel_ID',
  'Color',
  'Bodytype',
  'Gearbox',
  'Fuel_type']


categorical_df = data_df.loc[:,categorical_ids]

'''
연속형 데이터 전처리
'''

# continuous_df['Price'] = pd.to_numeric(continuous_df['Price'], errors='coerce')

# normalize - 정규화
# 각 열의 값을 정규화하여 평균은 0, 표준편차는 1로
continuous_df=(continuous_df-continuous_df.mean())/continuous_df.std()

'''
범주형 데이터 전처리
'''
categorical_df['Maker'] = categorical_df['Maker'].astype('category')
categorical_df['Genmodel'] = categorical_df['Genmodel'].astype('category')
categorical_df['Genmodel_ID'] = categorical_df['Genmodel_ID'].astype('category')
categorical_df['Color'] = categorical_df['Color'].astype('category')
categorical_df['Bodytype'] = categorical_df['Bodytype'].astype('category')
categorical_df['Gearbox'] = categorical_df['Gearbox'].astype('category')
categorical_df['Fuel_type'] = categorical_df['Fuel_type'].astype('category')


# 범주형 데이터 -> 숫자 코드 
cat_columns = categorical_df.select_dtypes(['category']).columns
categorical_df[cat_columns] = categorical_df[cat_columns].apply(lambda x: x.cat.codes)

# 최종 데이터 병합
data_df = pd.concat([id_df, continuous_df, categorical_df, image_name_df, viewpoint_df], axis=1)
# 결측값 제거
data_df.dropna(inplace=True)

data_df

,ID,Wheelbase,Height,Width,Length,Seat_num,Door_num,Entry_price,Year,First_release_year,...,Engine_size,Maker,Genmodel,Genmodel_ID,Color,Bodytype,Gearbox,Fuel_type,Image_name,Predicted_viewpoint
0,10_1$$2,2.476883,-0.113537,1.542787,2.520060,0.141542,-0.396578,6.996262,-2.406355,-0.760046,...,6.401685,5,81,0,8,11,0,8,Bentley$$Arnage$$2002$$Grey$$10_1$$2$$image_0.jpg,0
1,10_1$$3,2.476883,-0.113537,1.542787,2.520060,0.141542,-0.396578,6.996262,-2.406355,-0.760046,...,6.401685,5,81,0,2,11,0,8,Bentley$$Arnage$$2002$$Blue$$10_1$$3$$image_0.jpg,90
2,10_1$$4,2.476883,-0.113537,1.542787,2.520060,0.141542,-0.396578,7.132768,-2.172015,-0.760046,...,6.401685,5,81,0,7,11,0,8,Bentley$$Arnage$$2003$$Green$$10_1$$4$$image_0...,315
3,10_1$$5,2.476883,-0.113537,1.542787,2.520060,0.141542,-0.396578,7.132768,-2.172015,-0.760046,...,6.401685,5,81,0,8,11,0,8,Bentley$$Arnage$$2003$$Grey$$10_1$$5$$image_10...,0
4,10_1$$6,2.476883,-0.113537,1.542787,2.520060,0.141542,-0.396578,6.996262,-2.406355,-0.760046,...,6.401685,5,81,0,2,11,0,8,Bentley$$Arnage$$2002$$Blue$$10_1$$6$$image_0.jpg,315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173466,96_9$$521,0.005620,-0.635524,-0.796844,0.399052,0.141542,0.618468,-0.277897,-1.234652,-0.111250,...,0.153559,57,464,508,18,4,1,1,Volvo$$V50$$2007$$Silver$$96_9$$521$$image_0.jpg,225
173467,96_9$$523,0.005620,-0.594096,-0.796844,0.418421,0.141542,0.618468,-0.277897,-1.234652,-0.111250,...,0.674237,57,464,508,2,4,0,1,Volvo$$V50$$2007$$Blue$$96_9$$523$$image_0.jpg,315
173468,96_9$$524,0.005620,-0.594096,-0.796844,0.418421,0.141542,0.618468,-0.278443,-0.765971,-0.111250,...,0.674237,57,464,508,18,4,0,1,Volvo$$V50$$2009$$Silver$$96_9$$524$$image_0.jpg,135
173469,96_9$$525,0.005620,-0.635524,-0.796844,0.399052,0.141542,0.618468,-0.202272,-1.937674,-0.111250,...,0.674237,57,464,508,18,4,0,8,Volvo$$V50$$2004$$Silver$$96_9$$525$$image_0.jpg,135


### 데이터 부족한 클래스 제거

In [18]:
'''
데이터 부족한 클래스나 카테고리 제거, 데이터가 충분한 카테고리만 선택
'''
minimum_population = 100
values = (data_df.value_counts(subset=['Genmodel_ID'])>=minimum_population).values
codes = (data_df.value_counts(subset=['Genmodel_ID'])>=minimum_population).index
populated_codes = []
for i, v in enumerate(values):
  if v:
    populated_codes.append(int(codes[i][0]))

In [19]:
len(populated_codes)

264

In [20]:
data_df = data_df[data_df['Genmodel_ID'].isin(populated_codes)]
map = {}
for i,l in enumerate(data_df['Genmodel_ID'].unique()):
  map[l] = i
data_df['Genmodel_ID'] = data_df['Genmodel_ID'].map(map)
data_df

/tmp/ipykernel_2197662/4077603086.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['Genmodel_ID'] = data_df['Genmodel_ID'].map(map)


,ID,Wheelbase,Height,Width,Length,Seat_num,Door_num,Entry_price,Year,First_release_year,...,Engine_size,Maker,Genmodel,Genmodel_ID,Color,Bodytype,Gearbox,Fuel_type,Image_name,Predicted_viewpoint
18,10_3$$1,1.848684,1.767271,2.195248,1.914749,0.141542,0.618468,6.275508,0.874414,2.267669,...,5.360331,5,98,0,8,10,0,8,Bentley$$Bentayga$$2016$$Grey$$10_3$$1$$image_...,225
19,10_3$$2,1.848684,1.767271,2.195248,1.914749,0.141542,0.618468,6.275508,1.343095,2.267669,...,5.360331,5,98,0,4,10,0,8,Bentley$$Bentayga$$2018$$Brown$$10_3$$2$$image...,90
20,10_3$$3,1.848684,1.767271,2.195248,1.914749,0.141542,0.618468,6.275508,0.874414,2.267669,...,5.360331,5,98,0,18,10,0,8,Bentley$$Bentayga$$2016$$Silver$$10_3$$3$$imag...,225
21,10_3$$4,1.848684,1.767271,2.195248,1.914749,0.141542,0.618468,6.275508,1.343095,2.267669,...,2.756945,5,98,0,8,10,0,1,Bentley$$Bentayga$$2018$$Grey$$10_3$$4$$image_...,90
22,10_3$$6,1.848684,1.767271,2.195248,1.914749,0.141542,0.618468,6.275508,1.343095,2.267669,...,2.756945,5,98,0,1,10,0,8,Bentley$$Bentayga$$2018$$Black$$10_3$$6$$image...,315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173466,96_9$$521,0.005620,-0.635524,-0.796844,0.399052,0.141542,0.618468,-0.277897,-1.234652,-0.111250,...,0.153559,57,464,263,18,4,1,1,Volvo$$V50$$2007$$Silver$$96_9$$521$$image_0.jpg,225
173467,96_9$$523,0.005620,-0.594096,-0.796844,0.418421,0.141542,0.618468,-0.277897,-1.234652,-0.111250,...,0.674237,57,464,263,2,4,0,1,Volvo$$V50$$2007$$Blue$$96_9$$523$$image_0.jpg,315
173468,96_9$$524,0.005620,-0.594096,-0.796844,0.418421,0.141542,0.618468,-0.278443,-0.765971,-0.111250,...,0.674237,57,464,263,18,4,0,1,Volvo$$V50$$2009$$Silver$$96_9$$524$$image_0.jpg,135
173469,96_9$$525,0.005620,-0.635524,-0.796844,0.399052,0.141542,0.618468,-0.202272,-1.937674,-0.111250,...,0.674237,57,464,263,18,4,0,8,Volvo$$V50$$2004$$Silver$$96_9$$525$$image_0.jpg,135


In [21]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 166682 entries, 18 to 173470
Data columns (total 21 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   166682 non-null  object 
 1   Wheelbase            166682 non-null  float64
 2   Height               166682 non-null  float64
 3   Width                166682 non-null  float64
 4   Length               166682 non-null  float64
 5   Seat_num             166682 non-null  float64
 6   Door_num             166682 non-null  float64
 7   Entry_price          166682 non-null  float64
 8   Year                 166682 non-null  float64
 9   First_release_year   166682 non-null  float64
 10  Sales_amt            166682 non-null  float64
 11  Engine_size          166682 non-null  float64
 12  Maker                166682 non-null  int8   
 13  Genmodel             166682 non-null  int16  
 14  Genmodel_ID          166682 non-null  int64  
 15  Color                

### Train / Val / Test 데이터 분리

In [ ]:
# Genmodel_ID로 안묶은 train/val/test -> 64:16:20
from sklearn.model_selection import train_test_split

_ids = list(data_df['ID'])

train_set_ids, test_ids = train_test_split(_ids, test_size=0.2, random_state=2022, stratify=data_df['Genmodel_ID'])
train_ids, val_ids = train_test_split(train_set_ids, test_size=0.2, random_state=2022, stratify=data_df[data_df['ID'].isin(train_set_ids)]['Genmodel_ID'])

# 결과 확인
print(f"Train size: {len(train_ids)}")
print(f"Validation size: {len(val_ids)}")
print(f"Test size: {len(test_ids)}")

Train size: 106676
Validation size: 26669
Test size: 33337


In [95]:
# Genmodel_ID 별로 묶은 train/val/test

# from sklearn.model_selection import train_test_split

# def stratified_split_by_genmodel(data_df, test_size=0.2, val_size=0.2, random_state=2022):
#     # Unique Genmodel_ID 리스트 생성
#     unique_genmodels = data_df['Genmodel_ID'].unique()
    
#     # Genmodel_ID 기준으로 Train/Val/Test 분할
#     train_genmodels, temp_genmodels = train_test_split(
#         unique_genmodels, test_size=(val_size + test_size), random_state=random_state, stratify=None
#     )
#     val_genmodels, test_genmodels = train_test_split(
#         temp_genmodels, test_size=(test_size / (val_size + test_size)), random_state=random_state, stratify=None
#     )
    
#     # 각각의 Genmodel_ID에 해당하는 데이터 추출
#     train_df = data_df[data_df['Genmodel_ID'].isin(train_genmodels)]
#     val_df = data_df[data_df['Genmodel_ID'].isin(val_genmodels)]
#     test_df = data_df[data_df['Genmodel_ID'].isin(test_genmodels)]
    
#     # ID 리스트 추출
#     train_ids = list(train_df['ID'])
#     val_ids = list(val_df['ID'])
#     test_ids = list(test_df['ID'])
    
#     return train_df, val_df, test_df, train_ids, val_ids, test_ids

# # 실행
# train_df, val_df, test_df, train_ids, val_ids, test_ids = stratified_split_by_genmodel(data_df)

# # 결과 확인
# print(f"Train size: {len(train_df)}")
# print(f"Validation size: {len(val_df)}")
# print(f"Test size: {len(test_df)}")

Train size: 98278
Validation size: 35177
Test size: 33227


### Tabular 데이터 저장

In [24]:
'''
train_ids_~.pt
Train/Val/Test 별로 Adv_id
'''
addendum = '_all_views'
non_feature_columns = ['ID', 'Image_name', 'Predicted_viewpoint', 'Genmodel_ID', 'Sales_amt']

check_or_save(train_ids, join(FEATURES, f'train_ids{addendum}.pt'))
check_or_save(val_ids, join(FEATURES, f'val_ids{addendum}.pt'))
check_or_save(test_ids, join(FEATURES, f'test_ids{addendum}.pt'))

train_df = data_df.set_index('ID').loc[train_ids]
val_df = data_df.set_index('ID').loc[val_ids]
test_df = data_df.set_index('ID').loc[test_ids]

train_labels_all = list(train_df['Sales_amt'])
val_labels_all = list(val_df['Sales_amt'])
test_labels_all = list(test_df['Sales_amt'])

'''
labels_model_all_~.pt
Train/Val/Test 별로 Label(Sales_amt) 저장
'''
check_or_save(train_labels_all, join(FEATURES,f'labels_model_all_train{addendum}.pt'))
check_or_save(val_labels_all, join(FEATURES,f'labels_model_all_val{addendum}.pt'))
check_or_save(test_labels_all, join(FEATURES,f'labels_model_all_test{addendum}.pt'))

'''
dvm_features ~.csv
Train/Val/Test 별로 전처리된 특징 데이터 저장
non_feature_columns = ['ID', 'Image_name', 'Predicted_viewpoint', 'Genmodel_ID', 'Sales_amt'] 컬럼이 제외된 데이터만 저장
'''
check_or_save(train_df.loc[:,~train_df.columns.isin(non_feature_columns)],join(FEATURES,f'dvm_features_train_noOH{addendum}.csv'), index=False, header=False)
check_or_save(val_df.loc[:,~val_df.columns.isin(non_feature_columns)],join(FEATURES,f'dvm_features_val_noOH{addendum}.csv'), index=False, header=False)
check_or_save(test_df.loc[:,~test_df.columns.isin(non_feature_columns)],join(FEATURES,f'dvm_features_test_noOH{addendum}.csv'), index=False, header=False)

'''
dvm_full_features ~.csv
Train/Val/Test 별로 전처리된 모든 특징 데이터 저장
'''
check_or_save(train_df, join(FEATURES,f'dvm_full_features_train_noOH{addendum}.csv'), index=True, header=True)
check_or_save(val_df, join(FEATURES,f'dvm_full_features_val_noOH{addendum}.csv'), index=True, header=True)
check_or_save(test_df, join(FEATURES,f'dvm_full_features_test_noOH{addendum}.csv'), index=True, header=True)

lengths = [1 for i in range(len(continuous_df.columns)-1)]

if 'Genmodel_ID' in categorical_ids:
  categorical_ids.remove('Genmodel_ID')
max = list(data_df[categorical_ids].max(axis=0))
max = [i+1 for i in max]
lengths = lengths + max
check_or_save(lengths, join(FEATURES, f'tabular_lengths{addendum}.pt'))

Saving to /data/ephemeral/home/data/base_features/train_ids_all_views.pt
Saving to /data/ephemeral/home/data/base_features/val_ids_all_views.pt
Saving to /data/ephemeral/home/data/base_features/test_ids_all_views.pt


Saving to /data/ephemeral/home/data/base_features/labels_model_all_train_all_views.pt
Saving to /data/ephemeral/home/data/base_features/labels_model_all_val_all_views.pt
Saving to /data/ephemeral/home/data/base_features/labels_model_all_test_all_views.pt
Saving to /data/ephemeral/home/data/base_features/tabular_lengths_all_views.pt


In [25]:
train_df.columns

Index(['Wheelbase', 'Height', 'Width', 'Length', 'Seat_num', 'Door_num',
       'Entry_price', 'Year', 'First_release_year', 'Sales_amt', 'Engine_size',
       'Maker', 'Genmodel', 'Genmodel_ID', 'Color', 'Bodytype', 'Gearbox',
       'Fuel_type', 'Image_name', 'Predicted_viewpoint'],
      dtype='object')

### Image 데이터 경로 저장

In [26]:
bad_indices = []
for indx, row in data_df.iterrows():
    im_name = row['Image_name']
    split = im_name.split('$$')
    path = join(BASE, 'resized_DVM', split[0], split[1], split[2], split[3], im_name)
    if not os.path.exists(path):
        bad_indices.append(path)

In [27]:
'''
train, val, test 데이터셋에 대한 이미지 파일 경로 생성 및 저장
'''

def get_paths(df):
  paths = []
  for indx, row in df.iterrows():
      im_name = row['Image_name']
      split = im_name.split('$$')
      path = join(BASE, 'resized_DVM', split[0], split[1], split[2], split[3], im_name)
      paths.append(path)
  return paths

# For big dataset need to save only paths to load live
addendum = '_all_views'
train_df = pd.read_csv(join(FEATURES,f'dvm_full_features_train_noOH{addendum}.csv'))
val_df = pd.read_csv(join(FEATURES,f'dvm_full_features_val_noOH{addendum}.csv'))
test_df = pd.read_csv(join(FEATURES,f'dvm_full_features_test_noOH{addendum}.csv'))

for df, name in zip([train_df, val_df, test_df], ['train', 'val', 'test']):
  paths = get_paths(df)
  check_or_save(paths, join(FEATURES, f'{name}_paths{addendum}.pt'))

Saving to /data/ephemeral/home/data/base_features/train_paths_all_views.pt
Saving to /data/ephemeral/home/data/base_features/val_paths_all_views.pt
Saving to /data/ephemeral/home/data/base_features/test_paths_all_views.pt


# reorder features to categorical, numerical

In [28]:
# reorder features to categorical, numerical
for v in ['_all_views']:
    field_lengths_tabular = torch.load(join(FEATURES, f'tabular_lengths{v}.pt'))
    categorical_ids = []
    continous_ids = []
    for i in range(len(field_lengths_tabular)):
        if field_lengths_tabular[i] == 1:
            continous_ids.append(i)
        else:
            categorical_ids.append(i)
    print('Categorical Index: {}, '.format(len(categorical_ids)), categorical_ids)
    print('Numerical Index: {}, '.format(len(continous_ids)), continous_ids)

    reorder_ids = categorical_ids + continous_ids
    reorder_field_lengths_tabular = [field_lengths_tabular[i] for i in reorder_ids]
    check_or_save(reorder_field_lengths_tabular, join(FEATURES, f'tabular_lengths{v}_reordered.pt'),)

    for split in ['train', 'val', 'test']:
        data_tabular = pd.read_csv(join(FEATURES, f'dvm_features_{split}_noOH{v}.csv'), header=None)
        reorder_data_tabular = data_tabular.iloc[:, reorder_ids]
        check_or_save(reorder_data_tabular, join(FEATURES, f'dvm_features_{split}_noOH{v}_reordered.csv'), index=False, header=False)
    # for k in ['_0.1', '_0.01']:
    #     data_tabular = pd.read_csv(join(FEATURES, f'dvm_features_train_noOH{v}{k}_jittered_50.csv'), header=None)
    #     reorder_data_tabular = data_tabular.iloc[:, reorder_ids]
    #     check_or_save(reorder_data_tabular, join(FEATURES, f'dvm_features_train_noOH{v}{k}_jittered_50_reordered.csv'), index=False, header=False)

Categorical Index: 6,  [10, 11, 12, 13, 14, 15]
Numerical Index: 10,  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Saving to /data/ephemeral/home/data/base_features/tabular_lengths_all_views_reordered.pt
